# $R_S$ and $R_N$ barplots for each genome

TODO get this working!

In [1]:
%run "Header.ipynb"

In [12]:
import json
import skbio
from parse_sco import parse_sco

In [4]:
with open("matrix-jsons/seq2gene2codon2alignedcodons.json", "r") as jf:
    seq2gene2codon2alignedcodons = json.load(jf)

## _minfreq_ values

Currently the same as in the codon mutation notebook. I guess these should be a shared variable, ideally? But that might be too much work for its own good.

In [10]:
percentages = [2, 1, 0.5, 0.25, 0.1, 0.05, 0]

In [18]:
# Record R_S and R_N values for each sequence for each percentages, so we can separate computation from plotting
# somewhat
p2seq2vals = {}
for p in percentages:
    print(f"On percentage {p}%.")
    p2seq2vals[p] = {}
    
    # Code partially yanked from the mutation matrix notebook
    for seq in SEQS:
        print(f"\tOn seq {seq}.")
        
        singlent_synonymous_codon_mutation_ct = 0
        singlent_nonsynonymous_codon_mutation_ct = 0
        
        fasta = skbio.DNA.read("../seqs/{}.fasta".format(seq))
        df = parse_sco("../seqs/genes/{}.sco".format(seq))
        
        for gene_data in df.itertuples():
            
            for cpleft in range(gene_data.LeftEnd, gene_data.RightEnd + 1, 3):

                # Make note of the codon sequence and amino acid encoded by this codon in the "reference" genome.
                # (Keep in mind that the gene data in the .sco file uses 1-indexed coords, so we need to convert
                # accordingly.)
                codon_dna = fasta[cpleft - 1: cpleft + 2]
                if gene_data.Strand == "-":
                    codon_dna = codon_dna.reverse_complement()

                codon_seq = str(codon_dna)

                aligned_codons = seq2gene2codon2alignedcodons[seq][str(gene_data.Index)][str(cpleft)]
                num_aligned_codons = sum(aligned_codons.values())

                # If num_aligned_codons is 0, then it means no reads fully covered this codon. So we can't
                # "call" this codon as mutated.
                # 
                # Similarly, if the reference codon sequence isn't even included in the aligned codons (but if
                # there are still other codons included in the alignment???), we're likely
                # at a weird low-coverage portion of the alignment. Ignore these cases (there are 116 such
                # problematic codons as of writing this, mostly in edge_1671 -- something might be wrong with
                # the alignment, which I had to recover from a backup recently -- will rerun and check).
                if num_aligned_codons > 0 and codon_seq in aligned_codons:
                    
                    alt_codon_frac = (num_aligned_codons - aligned_codons[codon_seq]) / num_aligned_codons

                    # Use the value of minfreq depending on the outermost loop
                    if alt_codon_frac > (p / 100):

                        # Subset aligned_codons to just the alternate codons. I guess we could also just use "del".
                        alt_codons = {c: aligned_codons[c] for c in aligned_codons if c != codon_seq}

                        # Retrieve max-freq alternate codon.
                        # Based on https://stackoverflow.com/a/280156.
                        # (Note that if there's a tie, the result is arbitrary. As mentioned in paper.)
                        max_freq_alt_codon = max(alt_codons, key=alt_codons.get)
                        
                        # If the max freq alt codon is only a single nucleotide different from the original codon,
                        # update the singlent_* variables accordingly. Otherwise, ignore this case.
                        
                        num_diffs = 0
                        for pos in (0, 1, 2):
                            if max_freq_alt_codon[pos] != codon_seq[pos]:
                                num_diffs += 1
                        if num_diffs > 1:
                            continue
                        elif num_diffs == 1:
                            aa = str(codon_dna.translate())
                            aa2 = str(skbio.DNA(max_freq_alt_codon).translate())
                            if aa == aa2:
                                singlent_synonymous_codon_mutation_ct += 1
                            else:
                                singlent_nonsynonymous_codon_mutation_ct += 1
                        else:
                            # This should never happen, since we filtered out codon_seq from the alt_codons
                            # dict already.
                            raise ValueError(f"Seems like {max_freq_alt_codon} is identical to {codon_seq}???")
                            
        p2seq2vals[p][seq] = {}
        p2seq2vals[p][seq]["rs"] = singlent_synonymous_codon_mutation_ct
        p2seq2vals[p][seq]["rs"] = singlent_nonsynonymous_codon_mutation_ct

On percentage 2%.
	On seq edge_6104.
	On seq edge_1671.
	On seq edge_2358.


KeyboardInterrupt: 